In [1]:
!pip install numpy
!pip install opencv-python
!pip install tqdm
!pip install scikit-learn
!pip install matplotlib

In [2]:
import os

BASE_DIR = r"F:\DeeperForensics_Dataset"
SOURCE_DIR = os.path.join(BASE_DIR, "source_videos")  # Real videos
MANIPULATED_DIR = os.path.join(BASE_DIR, "manipulated_videos")  # Fake videos

def collect_videos(base_path):
    video_paths = []
    for root, _, files in os.walk(base_path):
        for file in files:
            if file.endswith('.mp4'):
                video_paths.append(os.path.join(root, file))
    return video_paths

real_videos = collect_videos(SOURCE_DIR)
fake_videos = collect_videos(MANIPULATED_DIR)

print(f"Found {len(real_videos)} real videos and {len(fake_videos)} fake videos.")


Found 48475 real videos and 11000 fake videos.


In [6]:
import cv2
import os

# Directory containing your video files
video_dir = "F:/DeeperForensics_Dataset/source_videos"
output_dir = "E:/Extracted_Frames_Angry"
emotion = "angry"  # Emotion filter


# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Function to resize the frame while preserving the aspect ratio
def resize_with_aspect_ratio(image, max_dim=512):
    height, width = image.shape[:2]

    # Calculate scaling factor to preserve aspect ratio
    if height > width:
        scale = max_dim / height
    else:
        scale = max_dim / width

    # Compute new dimensions
    new_width = int(width * scale)
    new_height = int(height * scale)

    # Resize image
    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)
    return resized_image

# Function to process a video
# Function to process a video
def extract_frames(video_path, output_path, fps=24, max_dim=512):
    cap = cv2.VideoCapture(video_path)
    video_name = os.path.splitext(os.path.basename(video_path))[0]

    # Check if the video was loaded successfully
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return

    # Video properties
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    if original_fps == 0:
        print(f"Error reading FPS for {video_path}")
        return

    # Time interval between frames to extract (in seconds)
    time_interval = 1 / fps

    # Initialize variables
    frame_count = 0
    extracted_count = 0
    current_time = 0.0  # Current timestamp to extract frame

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # End of video

        # Get the current frame's timestamp
        frame_time = frame_count / original_fps

        # Extract frame if it matches the desired time interval
        if frame_time >= current_time:
            frame_output_dir = os.path.join(output_path, video_name)
            os.makedirs(frame_output_dir, exist_ok=True)

            # Resize the frame while maintaining aspect ratio
            resized_frame = resize_with_aspect_ratio(frame, max_dim)

            # Save the resized frame
            frame_path = os.path.join(frame_output_dir, f"frame_{extracted_count:04d}.jpg")
            cv2.imwrite(frame_path, resized_frame)
            extracted_count += 1

            # Update the next extraction time
            current_time += time_interval

        frame_count += 1

    cap.release()
    print(f"Frames extracted from {video_path}: {extracted_count}")
    print(f"Original FPS: {original_fps}, Target FPS: {fps}, Duration: {frame_count / original_fps:.2f}s")


# Traverse video directory to locate videos with "angry" emotion
for root, dirs, files in os.walk(video_dir):
     if emotion in root.lower():  # Case-insensitive matching for "angry"
        for file in files:
            if file.endswith((".mp4", ".avi", ".mov")):  # Add other formats if necessary
                video_path = os.path.join(root, file)
                print(f"Processing {video_path}")
                extract_frames(video_path, output_dir, fps=24, max_dim=512)


Processing F:/DeeperForensics_Dataset/source_videos\M004\light_up\angry\camera_front\M004_light_up_angry_camera_front.mp4
Frames extracted from F:/DeeperForensics_Dataset/source_videos\M004\light_up\angry\camera_front\M004_light_up_angry_camera_front.mp4: 173
Original FPS: 30.0, Target FPS: 24, Duration: 7.23s
Processing F:/DeeperForensics_Dataset/source_videos\M004\light_up\angry\camera_right\M004_light_up_angry_camera_right.mp4
Frames extracted from F:/DeeperForensics_Dataset/source_videos\M004\light_up\angry\camera_right\M004_light_up_angry_camera_right.mp4: 173
Original FPS: 30.0, Target FPS: 24, Duration: 7.23s
Processing F:/DeeperForensics_Dataset/source_videos\M004\light_up\angry\camera_down\M004_light_up_angry_camera_down.mp4
Frames extracted from F:/DeeperForensics_Dataset/source_videos\M004\light_up\angry\camera_down\M004_light_up_angry_camera_down.mp4: 173
Original FPS: 30.0, Target FPS: 24, Duration: 7.23s
Processing F:/DeeperForensics_Dataset/source_videos\M004\light_up\an